# Lab 4, Reinforcement Learning

Machine Learning for Industry, Linköping University, Fall 2019.

Author: [Joel Oskarsson](mailto:joeos014@student.liu.se), Linköping University

## Introduction
The purpose of this lab is to get some practical experience with reinforcement learning and understand how an agent can learn from acting in an environment. 
Your will implement key parts of the Q-learning and REINFORCE algorithms. Details on these algorithms can be found in the lecture slides and the book by Sutton and Barton.

## Agent and Environment
The first step when applying reinforcement learning to a problem is to clearly define the environment, possible actions and rewards. 
In this lab we will work with a grid-world environment consisting of $H$ x $W$ tiles laid out in a 2-dimensional grid, as can be seen in figure 1.
<center>
    <img width="350" src="https://gitlab.liu.se/joeos014/figures/raw/master/rl/environment.png">
    <br>
    Figure 1: The grid-world environment
</center>

The agent can act by moving up, down, left or right. As a Markov decision process this is:

* State space: $\mathcal{S} = \left\{(y,x) | y \in \left\{0,1,\dots,H-1\right\}, x \in \left\{0,1,\dots,W-1\right\}\right\}$
* Action space: $\mathcal{A} = \left\{\text{up}, \text{down}, \text{left}, \text{right}\right\}$

Additionally:
* We assume the state space to be fully observable.
* The reward function is a deterministic function of the state and does not depend on the actions taken by the agent. Assume the agent gets the reward as soon as it moves to a state.
* The transition model is defined by the agent moving in the direction chosen with probability $(1-\beta)$. The agent might also slip and end up moving in the direction to the left or right of its chosen action, each with a probability $\beta/2$. This is illustrated in figure 2.
<center>
    <img width="350" src="https://gitlab.liu.se/joeos014/figures/raw/master/rl/transition_model.png">
    <br>
    Figure 2: The transition model
</center>

* The transition model is unknown to the agent, forcing us to resort to model-free approaches.
* The environment is episodic and all states with a non-zero reward are terminal (each episode ends when the agent receives **any** reward).
* Throughout this lab we are going to need integer representations of the different actions. For consistency we always use the following mapping: **{up: 0, right: 1, down: 2, left: 3}**.

Let's get started. We start by importing some neccesary Python packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as coll
import matplotlib.patches as patch

import scipy.linalg as lin
from keras import models, layers, optimizers, utils

# Set random seed
np.random.seed(42)

If there is some package that you have not used before it should be straightforward to install it using pip.

### Environment A
For our first environment we will choose $H=5$, $W=7$. 
This environment includes a reward of 10 in state $(2,5)$ and a reward of -1 in each of the states $(1,2), (2,2), (3,2)$.
We specify the rewards using a reward map in the form of a matrix with one entry for each state. States with no reward will simply have a matrix entry of 0.
The agent starts each episode in the state $(2,0)$ and for the transition model we set $\beta = 0$, so all transitions are completely deterministic.

In [ ]:
H = 5
W = 7

reward_map = np.zeros(shape=(H,W))
reward_map[2,5] = 10.
reward_map[1:4,2] = -1.

The function <tt>vis_environment</tt> can be used to visualize the environment and learned policy. You will not have to modify this function, but read the docstring to familiarize yourself with how it can be used.

In [ ]:
TILE_SIZE = 0.9
ARROWS = ["↑", "→", "↓", "←"]

def vis_environment(reward_map, q_table=None, policy=None, title=None):
    """
    Visualize an environment with rewards. 
    If a Q-table is given the Q-values for all actions are displayed on the edges of each tile.
    If a policy is given also visualize the chosen action for each state.
    
    Args:
        reward_map: a HxW numpy array containing the reward given at each state
        q_table (optional): a HxWx4 numpy array containing Q-values for each state-action pair
        policy (optional): policy object that implements an act method
        title (optional): figure title
    """
    
    fig, ax = plt.subplots(1, figsize=(15,10))
    
    tiles = []
    size_y, size_x = reward_map.shape
    
    max_val = np.max(reward_map)
    min_val = np.min(reward_map)
    
    if not (q_table is None):
        max_val = max(max_val, np.max(q_table))
        min_val = min(min_val, np.min(np.max(q_table, axis=2)))
    
    for i_x in range(size_x): 
        for i_y in range(size_y): 
            x = i_x - 0.5*TILE_SIZE
            y = i_y - 0.5*TILE_SIZE
            
            tile_value = 0.
            tile_reward = reward_map[i_y, i_x]
            if tile_reward != 0.:
                tile_value = tile_reward
                
                # Write out reward
                ax.text(i_x, i_y, s="R={:.2f}".format(tile_reward), 
                        verticalalignment="center", horizontalalignment="center", fontsize=12) # reward
            elif not (q_table is None):
                tile_value = np.max(q_table[i_y,i_x])
                
                # Direction Q-values
                ax.text(i_x, i_y-0.5*TILE_SIZE, s="{:.2f}".format(q_table[i_y,i_x,0]), 
                        verticalalignment="top", horizontalalignment="center", fontsize=12) # up
                ax.text(i_x, i_y+0.5*TILE_SIZE, s="{:.2f}".format(q_table[i_y,i_x,2]), 
                        verticalalignment="bottom", horizontalalignment="center", fontsize=12) # down
                ax.text(i_x-0.5*TILE_SIZE, i_y, s="{:.2f}".format(q_table[i_y,i_x,3]), 
                        verticalalignment="center", horizontalalignment="left", fontsize=12) # left
                ax.text(i_x+0.5*TILE_SIZE, i_y, s="{:.2f}".format(q_table[i_y,i_x,1]), 
                        verticalalignment="center", horizontalalignment="right", fontsize=12) # right
            
            # Set tile color based on value
            if tile_value >= 0:
                color_scale = tile_value/max_val if max_val > 0 else 0.0
                tile_color = (1.0-color_scale,1.0,1.0-color_scale)
            else:
                color_scale = abs(tile_value/min_val)
                tile_color = (1.0,1.0-color_scale,1.0-color_scale)
            
            tiles.append(patch.Rectangle((x,y),TILE_SIZE,TILE_SIZE, facecolor=tile_color))
        
            
            if policy and (tile_reward == 0.):
                # Show policy decisions
                policy_action = policy.act(i_y, i_x)
                ax.text(i_x, i_y, s=ARROWS[policy_action], verticalalignment="center", 
                        horizontalalignment="center", fontsize=30)
    
    tile_coll = coll.PatchCollection(tiles, match_original=True, edgecolor="#787878")
    ax.add_collection(tile_coll)
    ax.set_xlim(-0.5, size_x-0.5)
    ax.set_ylim(-0.5, size_y-0.5)
    plt.gca().invert_yaxis()
    
    if title:
        plt.title(title, fontsize=15)
    
    plt.show()

<h2 style="background-color:orange;margin-bottom:0px;">Task 0.1: Visualize Rewards</h2>

Use the <tt>vis_environment</tt> function to visualize the rewards.

In [ ]:
vis_environment(reward_map, title="Rewards")

## Section 1: Q-learning
We will now use Q-learning to make the agent learn a policy for acting in this environment.
Recall that the Q-learning update is defined as:
$$
Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha \left(R_{t+1} + \gamma \max_{a \in \mathcal{A}} Q_t(S_{t+1},a) - Q(S_t,A_t) \right) 
$$
for taking the action $A_t$ in state $S_t$, ending up at state $S_{t+1}$ and receiving reward $R_{t+1}$.
Unless stated otherwise we will use the values $\alpha = 0.1$ and $\gamma=0.95$ throughout this lab.

When implementing Q-learning the estimated values of $Q(S,A)$ are commonly stored in a data-structured called Q-table. 
This is nothing but a tensor with one entry for each state-action pair.
Since we have a $H$ x $W$ environment with 4 actions we can use a 3D-tensor of dimensions $H$ x $W$ x 4 to represent our Q-table. 

<h2 style="background-color:orange;margin-bottom:0px;">Task 1.1: Define Q-table</h2>

* Define the Q-table as a numpy array with the shape described above. Initialize all Q-values to 0.
* Use the <tt>vis_environment</tt> function to visualize the initial state of the Q-table.

In [ ]:
q_table = np.zeros(shape=(H,W,4))
vis_environment(reward_map, q_table=q_table, title="Initial Q-Table")

Next let's define some policies we are going to need. Since Q-learning is an off-policy algorithm we will use an $\epsilon$-greedy exploration policy.

<h2 style="background-color:orange;margin-bottom:0px;">Task 1.2: Implement Policies</h2>

$\newcommand{\argmax}{\mathop{\mathrm{argmax}}}$
Below you will find class definitions for classes implementing a greedy policy and an $\epsilon$-greedy policy. Your task is to implement the <tt>act</tt> methods of these classes.

* Implement the <tt>act</tt> method of <tt>GreedyPolicy</tt> to act greedily with respect to the Q-table. In some states there might be multiple actions with the same Q-value. To speed up training it is a good idea to then sample uniformly from the set $\argmax_{a \in \mathcal{A}} Q(S,a)$.
* Implement the <tt>act</tt> method of <tt>EpsilonGreedyPolicy</tt>. Recall that an $\epsilon$-greedy policy acts randomly with probability $\epsilon$ and else acts greedily.

In [ ]:
class GreedyPolicy:
    """
    Policy for acting greedily with respect to a Q-table.
    """
    def __init__(self, q_table):
        """
        Initialize a new GreedyPolicy acting with respect to given Q-table.
        
        Args:
            q_table: Q-table to derive greedy policy from
        """
        self.q = q_table
        
    def act(self, y, x):
        """
        Get a greedy action for state (y,x) from the policy.
        Args:
            y: state y coordinate
            x: state x coordinate
            
        Returns:
             An action (integer in {0,1,2,3}) according to the policy.
        """
        
        # Your code here
        
        # Recall to sample uniformly from argmax set
        
        Q_s = self.q[y,x]
        argmax = np.argwhere(Q_s == np.amax(Q_s))[:,0]
        argmax_size = argmax.shape[0]

        if argmax_size == 1:
            return argmax[0]
        else:
            return argmax[np.random.randint(0,argmax_size)]

In [ ]:
class EpsilonGreedyPolicy(GreedyPolicy):
    """
    Policy for acting epsilon-greedily with respect to a Q-table.
    """
    def __init__(self, q_table, epsilon):
        """
        Initialize a new EpsilonGreedyPolicy.
        
        Args:
            q_table: Q-table to derive greedy policy from
            epsilon: probability to act randomly
        """
        self.epsilon = epsilon
        super(EpsilonGreedyPolicy, self).__init__(q_table)
        
    def act(self, y, x):
        """
        Get an action for state (y,x) from the policy .
        Args:
            y: state y coordinate
            x: state x coordinate
            
        Returns:
             An action (integer in {0,1,2,3}) according to the policy.
        """
        
        # Your code here
        
        # You can use super(EpsilonGreedyPolicy, self).act(y, x) to call act from GreedyPolicy
        
        trial = np.random.binomial(1, self.epsilon)
        
        if trial == 1:
            # Act randomly
            return np.random.randint(0, 4)
        else:
            # Act greedily
            return super(EpsilonGreedyPolicy, self).act(y, x)

Now that we have a policy to use for exploration, let's implement the main Q-learning algorithm.

<h2 style="background-color:orange;margin-bottom:0px;">Task 1.3: Implement Q-learning</h2>

The function <tt>q_learning</tt> should run one episode of the agent acting in the environment and update the Q-table accordingly. A transition model taking $\beta$ as input is implemented for you in the function <tt>transition</tt>.

* Implement the transition to new states using the given exploration policy and transition model.
* Implement the Q-table update based on the Q-learning update equation.
* Make the <tt>q_learning</tt> function return the episode reward and the sum of the temporal-difference correction terms $\left(R_{t+1} + \gamma \max_{a \in \mathcal{A}} Q_t(S_{t+1},a) - Q(S_t,A_t) \right)$ for all steps in the episode.

In [ ]:
# Useful constants
ACTION_DELTAS = np.array([
    [-1, 0], # up
    [0, 1], # right
    [1, 0], # down
    [0, -1], # left
])

def transition_model(state, action, beta, size_y, size_x):
    """
    Computes the new state after given action is taken. 
    The agent will follow the action with probability (1-beta) and slip ot the right or left with probability beta/2 each.

    Args:
        state: the state (y,x) the agent is currently in
        action: which action the agent takes (in {0,1,2,3})
        beta: probability of the agent slipping to the side when trying to move
        size_y: size of the state space in the y-dimension 
        size_x: size of the state space in the x-dimension 

    Returns:
        The new state after the action has been taken.
    """
    # Sample from categorical distribution
    delta = np.random.choice([-1, 0, 1], p=[0.5*beta, (1-beta), 0.5*beta])
    final_action = (action + delta + 4) % 4

    new_state = state + ACTION_DELTAS[final_action]
    new_state = np.clip(new_state, 0, [size_y-1, size_x-1]) # Clip to environment size
    return new_state


In [ ]:
def q_learning(q_table, reward_map, exploration_policy, start_state, alpha=0.1, gamma=0.95, beta=0.0):
    """
    Perform one episode of Q-learning. The agent should move around in the 
    environment using the given transition model and update the Q-table.
    The episode ends when the agent reaches a terminal state.
    
    Args:
        q_table: Q-table to update Q-values in
        reward_map: matrix containing the reward for each state in the state space
        exploration_policy: object of the policy class implementing an act-method to be used for exploring the state space
        start_state: numpy array with two entries, describing the starting position of the agent
        alpha (optional): learning rate
        gamma (optional): discount factor
        beta (optional): slipping factor
        
    Returns:
        reward: reward received in the episode
        correction: temporal difference correction term
    """
    
    # Your code here
    

<h2 style="background-color:orange;margin-bottom:0px;">Task 1.4: Learn !</h2>

* Run 1000 episodes of Q-learning. Use an $\epsilon$-greedy policy with $\epsilon=0.5$ for exploration.  
* After episode 10, 100 and 1000, visualize the Q-table and a greedy policy derived from it (use the <tt>vis_environment</tt> function).
* Answer the questions below:
    1. What has the agent learned after the first 10 episodes?
    2. Is the final greedy policy (after 1000 episodes) optimal? Why / Why not?
    3. Does the agent learn that there are multiple paths to get to the positive reward? If not, what could be done to make the agent learn this ?

In [ ]:
q_table = np.zeros(shape=(H,W,4))

# Define policies
expl_policy = EpsilonGreedyPolicy(q_table, 0.5)
greedy_policy = GreedyPolicy(q_table)

# Your code here


### Answers:

Your answers here

## Section 2: New Environments
In this section, we consider some new environments and investigate the impact of changing different parameters. All environments still fit within the previous grid-world definition.

### Environment B
This is a $7 \times 8$ environment where the top and bottom rows have a negative reward. 
In this environment the agent starts each episode in the state (3, 0) and the transition model is deterministic ($\beta = 0$).
There are two positive rewards, of 5 and 10. 
The reward of 5 is easily reachable, but the agent has to navigate around the first reward in order to find the reward worth 10. 
The reward map is defined below.

In [ ]:
H = 7
W = 8
reward_map = np.zeros(shape=(H,W))

reward_map[0,:] = -1.
reward_map[6,:] = -1.
reward_map[3,4] = 5.
reward_map[3,7] = 10.

vis_environment(reward_map)

Below you will find the function <tt>plot_training</tt> that plots reward and correction. 
You will not have to modify this function, but read the docstring to familiarize yourself with how it can be used. 
Since the values change with very high frequency <tt>plot_training</tt> applies moving-average smoothing to the data.


In [ ]:
def plot_training(rewards, corrections, title=None):
    """
    Plot curves for reward and correction based on Q-learning episodes.
    The curves are smoothed using a moving-average window.
    
    Args:
        rewards: a (numpy) array containing the received reward for each episode
        corrections: a (numpy) array containing the temporal difference correction term for each episode
        title (optional): title to display above the plots
    """
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    n = 100
    
    cummulative_reward = np.cumsum(rewards)
    reward_ma = (cummulative_reward[n:] - cummulative_reward[:-n])/n
    
    ax[0].plot(reward_ma)
    ax[0].set_title("Reward", fontsize=15)
    ax[0].set_xlabel("Episode")
    ax[0].set_ylabel("Reward")
    
    cummulative_corr = np.cumsum(corrections)
    corr_ma = (cummulative_corr[n:] - cummulative_corr[:-n])/n
    ax[1].plot(corr_ma)
    ax[1].set_title("Correction", fontsize=15)
    ax[1].set_xlabel("Episode")
    ax[1].set_ylabel("Correction")
    
    if title:
        plt.suptitle(title, fontsize=15)
    
    plt.show()

<h2 style="background-color:orange;margin-bottom:0px;">Task 2.1: Changing $\gamma$ and $\epsilon$</h2>

Let's now investigate how the $\gamma$ and $\epsilon$ parameters impact the learned policy.

* Fix $\epsilon = 0.5$ and for each $\gamma$ in $\{0.5, 0.75, 0.95\}$ train for 30000 episodes (this might take a while). Remember to reset the Q-table between runs.
* For each value of $\gamma$, use <tt>plot_training</tt> to plot the reward and correction and use <tt>vis_environment</tt> to visualize a greedy policy based on the learned Q-table.
* Repeat with $\epsilon = 0.1$.
* Explain your observations.

In [ ]:
# Your code here


In [ ]:
# Your code here


### Answers:

Your answers here

### Environment C
This is a smaller, $3 \times 6$ environment. 
Here the agent starts each episode in the state $(2,0)$.
The reward map is defined below.
In the following task we will experiment with different values of $\beta$ for this environment.

In [ ]:
H = 3
W = 6
reward_map = np.zeros(shape=(H,W))

reward_map[2,1:5] = -1.
reward_map[2,5] = 10.

vis_environment(reward_map)

<h2 style="background-color:orange;margin-bottom:0px;">Task 2.3: Probabilistic Transition Model</h2>

Let's investigate how different values of $\beta$ impact what the agent learns. For this experiment set $\gamma = 0.6$.

* For each value of $\beta$ in {0, 0.2, 0.4, 0.66} train the agent 10000 episodes using an $\epsilon$-greedy policy with $\epsilon = 0.5$. Remember to reset the Q-table between runs.
* Use <tt>vis_environment</tt> to visualize a greedy policy based on the learned Q-table for each of the values of $\beta$.
* Explain how the greedy policy changes with different values of $\beta$ and why.

In [ ]:
# Your code here


### Answers:
    
Your answers here

## Section 3: REINFORCE
In previous sections the policy was implicitly defined by estimated Q-values. 
Now we will take a different approach and learn the policy directly using function estimation.
For this we look to the area of policy gradient methods.
In particular we will implement the REINFORCE algorithm.

Recall that in policy gradient methods we consider the policy $\pi(A|S, \theta)$ to be a distribution over the action space, parameterized by $\theta$ and conditioned on a state $S$. 
The idea is then to optimize the policy using gradient ascent.
Let the performance of the policy parametrized by $\theta$ be 
$$
J(\theta) = v_{\pi_\theta}(s_0)
$$
that is, the value function of the starting state.
We are then interested in the gradient of $J$ w.r.t. $\theta$, which can be shown to be
$$
\nabla_\theta J(\theta) = \mathbb{E}_{\pi}\left[G_t \nabla_\theta \ln(\pi(A_t|S_t, \theta))\right]
$$
where
$$
G_t = \sum_{k=t+1}^{T} \gamma^{k-t-1}R_k
$$
for an episode of length $T$. 

The REINFORCE algorithm approximates the expectation in the gradient by sampling from the policy and performing stochastic gradient ascent. 
The sampling is performed by following the policy $\pi_\theta$ for one episode ("rolling out" the policy). 
This results in a trajectory, a sequence of states, actions and rewards on the form $S_0, A_0, R_1, S_1, \dots, S_{T-1}, A_{T-1}, R_T$.
Gradient ascent is then performed using the update rule
$$
\theta \leftarrow \theta + \alpha \gamma^t G_t \nabla_\theta \ln(\pi(A_t |S_t, \theta))
$$
for each $t = 0, 1, \dots, T-1$, where $\alpha$ is the learning rate.
Note that all future rewards are needed to compute $G_t$, so the entire episode has to be completed before any gradient ascent step can be taken.

We want to implement REINFORCE by parametrizing $\pi_\theta$ as a neural network. $\theta$ is then all trainable weights in the network. In supervised settings neural networks are often trained with stochastic gradient descent (or some extension thereof), following the update rule
$$
\theta \leftarrow \theta - \alpha w \nabla_\theta L(y, \hat{y})
$$
where $L$ is the loss function, $\alpha$ the learning rate, $y$ the ground truth and $\hat{y}$ the network output. 
$w$ is a sample-specific weight, usually assumed to just be 1 (all training samples are equally important).
Note that this is very similar to the REINFORCE update rule and in fact we can get exactly the REINFORCE update by:
* Using a cross-entropy loss $L(y, \hat{y}) = - \sum_i y_i \ln(\hat{y}_i)$
* Letting $y$ be a vector with 1 at the position corresponding to $A_t$ and zero everywhere else ("one-hot"-encoded).
* Feeding $S_t$ as input to the network.
* Setting the sample weight $w = \gamma^t G_t$.

Keeping in mind that the network outputs $\hat{y}_a = \pi(a | S_t, \theta)$, the choices above result in the network being trained with the REINFORCE update rule (insert the values into the stochastic gradient descent formulation to see this).
To speed up the computation we will also use batch gradient descent instead of training on each state-action pair individually.

Throughout this section we use a deterministic transition model ($\beta = 0$) and let $\gamma = 0.95$.

### Environment D
For this section we consider a $4 \times 4$ grid. 
This time around we want to teach the agent to navigate to a goal in **any** position.
The agent will start in a random position and it will be told the position of the goal.
The agent receives a reward of 5 when it reaches the goal.

In order to make the agent adapt its policy to reach the goal we need a way to tell the agent where the goal is. 
Since our agent doesn't have any memory mechanism, we provide the goal coordinates as part of the state at every timestep.
This increases the dimensionality of our state space to 4, now consisting of vectors $(y_{\text{agent}}, x_{\text{agent}}, y_{\text{goal}}, x_{\text{goal}})$. The actions of the agent can however only impact its own position, so there is no way to move to a part of the state space with different values of $y_{\text{goal}}$ and $x_{\text{goal}}$ in a single episode. Note the following:
* The agent has to discover that the so-called goal position receives the highest reward, which makes it the destination.
* The agent has to learn a policy to reach the destination.
* The policy has to depend on the destination, because the goal position will vary between episodes. This will make the agent able to reach previously unseen goal positions.

Since we only have a single reward we do not specify a reward map. Instead the goal coordinates are given to the neccesary functions.

In [ ]:
# Environment size
H = 4
W = 4

Since we are going to be working with a probabilistic policy in this section we will need some new ways of visualizing it. 
Below you'll find the function <tt>vis_prob</tt> that can be used to visualize Environment D with different goal positions and probabilistic policies.
As previously, you will not have to make any changes to this function, but read the docstring.

In [ ]:
TILE_SIZE = 0.9
ARROWS = ["↑", "→", "↓", "←"]

def vis_prob(goal, policy=None, title=None, size=(H,W)):
    """
    Visualize an environment with a single goal state. 
    If a policy is given also visualize the chosen action for each state and the probabilities of each action.
    
    Args:
        goal: coordinates of the goal state, array with 2 entries
        policy (optional): policy object that implements an act method and a get_action_dist method
        title (optional): figure title
        size (optional): dimensions of the environment, in the form (H,W)
    """
   
    fig, ax = plt.subplots(1, figsize=(15,10))
    
    tiles = []
    size_y, size_x = size
   
    for i_x in range(size_x): 
        for i_y in range(size_y): 
            x = i_x - 0.5*TILE_SIZE
            y = i_y - 0.5*TILE_SIZE
            
            is_goal = (i_x == goal[1]) and (i_y == goal[0])
            tile_color = (1.0,1.0,1.0)
            
            if is_goal:
                # Write out reward
                ax.text(i_x, i_y, s="G", 
                    verticalalignment="center", horizontalalignment="center", fontsize=12) # reward
                tile_color = (0.0,1.0,0.0)
                
            elif policy:
                # Show policy decisions
                action_dist = policy.get_action_dist(i_y, i_x, *goal)
                action = np.argmax(action_dist)
                ax.text(i_x, i_y, s=ARROWS[action], verticalalignment="center", 
                        horizontalalignment="center", fontsize=30)
                
                # Direction probabilities
                ax.text(i_x, i_y-0.5*TILE_SIZE, s="{:.2f}".format(action_dist[0]), 
                        verticalalignment="top", horizontalalignment="center", fontsize=12) # up
                ax.text(i_x, i_y+0.5*TILE_SIZE, s="{:.2f}".format(action_dist[2]), 
                        verticalalignment="bottom", horizontalalignment="center", fontsize=12) # down
                ax.text(i_x-0.5*TILE_SIZE, i_y, s="{:.2f}".format(action_dist[3]), 
                        verticalalignment="center", horizontalalignment="left", fontsize=12) # left
                ax.text(i_x+0.5*TILE_SIZE, i_y, s="{:.2f}".format(action_dist[1]), 
                        verticalalignment="center", horizontalalignment="right", fontsize=12) # right
           
            tiles.append(patch.Rectangle((x,y),TILE_SIZE,TILE_SIZE, facecolor=tile_color))
   
    tile_coll = coll.PatchCollection(tiles, match_original=True, edgecolor="#787878")
    ax.add_collection(tile_coll)
    ax.set_xlim(-0.5, size_x-0.5)
    ax.set_ylim(-0.5, size_y-0.5)
    plt.gca().invert_yaxis()
    
    if title:
        plt.title(title, fontsize=15)
    
    plt.show()

We will implement the policy network using [keras](https://keras.io/). 
Below you will find the definition of a small neural network with two hidden layers of 32 units each. 
The network takes 4 inputs (agent coordinates and goal coordinates) and has 4 outputs, allowing it to represent a distribution over the possible actions.

In [ ]:
model = models.Sequential([
    layers.Dense(32, input_shape=(4,)),
    layers.Dense(32),
    layers.Dense(4),
    layers.Softmax(),
])

opt = optimizers.SGD(learning_rate=0.001) # Keep learning rate at 0.001
model.compile(optimizer=opt, loss="categorical_crossentropy")

init_weights = model.get_weights() # Save initial weights for resetting later

<h2 style="background-color:orange;margin-bottom:0px;">Task 3.1: Deep Policy</h2>

To make it easy to work with, we wrap our neural network in a policy class. 
Below you will find the outline of the <tt>DeepPolicy</tt> class. 
Similarly to the earlier policies it has an <tt>act</tt> method, but <tt>DeepPolicy</tt> also has functions for getting the estimated distribution over actions (<tt>get_action_dist</tt>) and training the network on a rolled out trajectory (<tt>train</tt>).

* Implement <tt>get_action_dist</tt> according to its docstring.
* Implement <tt>act</tt> according to its docstring.
* Implement <tt>train</tt> according to its docstring.

Note that you can use the [predict](https://keras.io/models/model/#predict) method on the keras model to perform one forward pass through the network. You can use the [train_on_batch](https://keras.io/models/model/#train_on_batch) method on the keras model to train the network. Remember to use the <tt>sample_weight</tt> argument.

In [ ]:
class DeepPolicy:
    """
    Policy for acting using a deep neural network model.
    """
    
    def __init__(self, model, gamma=0.95):
        """
        Initialize a new DeepPolicy acting with respect to the given model.
        
        Args:
            model: keras model to use for estimating action probabilities
            gamma (optional): dicounting factor
        """
        
        self.network = model
        self.gamma = gamma
        
    def get_action_dist(self, y, x, goal_y, goal_x):
        """
        Get the estimated distribution over actions for the given state description.
        
        Args:
            y: agent y coordinate
            x: agent x coordinate
            goal_y: goal y coordinate
            goal_x: goal x coordinate
            
        Returns:
            A 4-entry array containing the probabilities of each action.
        """
        
        # Your code here
        
        net_input = np.array([[y, x, goal_y, goal_x]])
        return self.network.predict(net_input)[0]
        
    def act(self, y, x, goal_y, goal_x):
        """
        Get an action for the given state from the policy.
        
        Args:
            y: agent y coordinate
            x: agent x coordinate
            goal_y: goal y coordinate
            goal_x: goal x coordinate
            
        Returns:
             An action (integer in {0,1,2,3}) sampled from the estimated action distribution.
        """
        
        # Your code here
        
        # Hint: Use get_action_dist
        action_dist = self.get_action_dist(y, x, goal_y, goal_x)
        
        # Sample from categorical distribution
        action = np.random.choice(np.arange(4), p=action_dist)
        return action
    
    def train(self, states, actions, rewards, goal_y, goal_x):
        """
        Train the policy network on a rolled out trajectory.
        States, actions and rewards must all have the same length.
        
        Args:
            states: array of states visited throughout the trajectory, the first entry being the starting state
            actions: array of actions taken throughout the trajectory
            rewards: array of rewards received throughout the trajectory
            goal_y: goal y coordinate
            goal_x: goal x coordinate
        """
        
        # Your code here
        
        length = actions.size
        
        # Construct batch for training
        inputs = np.append(states, 
                           np.repeat([[goal_y, goal_x]], length, axis=0), 
                           axis=1)
        targets = utils.to_categorical(actions, num_classes=4)
        
        # Sample weights 
        discounting = np.power(self.gamma, np.arange(length))
        reward_matrix = lin.hankel(rewards)
        weighting = reward_matrix@discounting
        
        # Train on batch
        self.network.train_on_batch(x=inputs, y=targets, 
                                    sample_weight=weighting)

Next we'll instantiate our newly implemented <tt>DeepPolicy</tt>.

In [ ]:
deep_policy = DeepPolicy(model)

As with Q-learning we will use a function to run one episode of training in the environment. 
This time the function <tt>reinforce_episode</tt> below is completely implemented for you.
It is very similar to <tt>q_learning</tt>, but note that we here train on entire trajectories rather than single state transitions.
This means that all training happens at the end of each episode.

In [ ]:
def reinforce_episode(policy, goal, beta=0.0):
    """
    Rolls out a trajectory in the environment until the goal is reached.
    Then trains the policy using the collected states, actions and rewards.
    
    Args:
        policy: deep policy to follow and train
        goal: coordinates of the goal state, array with 2 entries
        beta (optional): probability of slipping in the transition model
    """
    
    # Randomize starting position
    cur_pos = goal
    while np.all(cur_pos == goal):
        cur_pos = np.random.randint(4, size=2)
    
    states = []
    actions = []
    rewards =  []
    
    steps = 0
    while True:
        # Follow policy
        action = policy.act(cur_pos[0], cur_pos[1], *goal)
        
        # Store state
        states.append(cur_pos)

        # Take action
        new_pos = transition_model(cur_pos, action, beta, H, W)
        steps += 1
        
        # 5 reward if the goal has been reached
        reward = int(np.all(new_pos == goal))*5.
        
        # Prevent getting stuck and/or training on unneccesarily long episodes
        if steps >= 200:
            break
        
        # Store Action, Reward
        actions.append(action)
        rewards.append(reward)
        
        cur_pos = new_pos

        if reward != 0.:
            # Train network
            policy.train(np.array(states), np.array(actions), 
                         np.array(rewards), *goal)
            break

<h2 style="background-color:orange;margin-bottom:0px;">Task 3.2: Random Goal Position</h2>

In this task we will use 8 goal positions for training and then validate on the remaining 8 possible goal positions. 
Each training episode should use a random goal position from <tt>TRAIN_GOALS</tt>.
Much of the code is implemented for you below.

* Run 3000 episodes of training using REINFORCE with a random goal position from <tt>TRAIN_GOALS</tt> for each episode. Note that we are now performing back-propagation in a neural network in each episode. Even though the network is small, this is quite computationally heavy and might take some time.
* Use the goal positions in <tt>VAL_GOALS</tt> for validation. Visualize the policy using <tt>vis_prob</tt> for these goal positions before and after training.
* Answer the questions below:
    1. Interpret the different action probabilities. In particular consider states where 
        1. There are multiple optimal actions.
        2. The agent prefers a suboptimal action (if such exists).
    2. What would happen if we applied Q-learning to this task?

In [ ]:
TRAIN_GOALS = [(0,0), (0,2), (1,0), (1,3), (2,0), (2,1), (3,1), (3,2),]
VAL_GOALS = [(0,1), (0,3), (1,1), (1,2), (2,2), (2,3), (3,0), (3,3),]

model.set_weights(init_weights)

def show_validation(policy, title):
    for g in VAL_GOALS:
        vis_prob(g, policy=policy, title=title)   

show_validation(deep_policy, "Before training")

# Your code here
      

### Answers:

Your answers here

<h2 style="background-color:orange;margin-bottom:0px;">Task 3.3: Training on Top Row</h2>

Now let's see what happens if the agent only trains with goals from the same row. 
In the <tt>TRAIN_GOALS</tt> given below all goal positions are on the top row.
We validate on three goal positions on the rows below.
We can of course validate the policy on more goal positions, but these three should be enough to draw some conclusions.

* Repeat the task 3.2 but with the <tt>TRAIN_GOALS</tt> and <tt>VAL_GOALS</tt> given below.
* Answer the questions below:
    1. Has the agent learned a good policy? Why / Why not?
    2. Compare this with the results from the task 3.2. 

In [ ]:
TRAIN_GOALS = [(0,0), (0,1), (0,2), (0,3), ]
VAL_GOALS = [(3,0), (1,3), (2,2),]

model.set_weights(init_weights)

show_validation(deep_policy, "Before training")

# Your code here
      

### Answers:

Your answers here

## Submission
* After you have completed all tasks, hand in the notebook file (Lab4.ipynb) through Lisam.
* Make sure that all results that you want to submit are in the notebook (run all cells that produce plots, etc.). 
* Make sure the submitted code reproduces your results. You can run the entire notebook from the start by selecting *Run -> Run all cells*.

## Extra Tasks
If you are interested in experimenting more with reinforcement learning you will find some ideas for extensions to this lab below. 
These are not part of the course and do not have to be handed in anywhere.

1. The REINFORCE implementation in section 3 is quite slow. 
One reason for this is that each step in the episode requires a forward pass through the network. 
This could be greatly improved by running multiple episodes in parallel, performing the forward pass on an entire batch of states. 
2. Try using Q-learning for the randomized goal task in Environment D. Compare it to using REINFORCE. 
3. The REINFORCE algorithm suffers from high variance. One way to help with this is to introduce a baseline (see section 13.4 in the book). 
Implement a baseline function parametrized as a second neural network, jointly trained with the policy network.
4. Extend the above approach to a full Actor-Critic method.
5. Beyond this notebook a nice platform for experimenting with reinforcement learning is [Open AI Gym](https://gym.openai.com/). 
It includes a number of different environments and can interface with implementations of most well-known RL algorithms. 
A good place to start is [Spinning up in Deep RL](https://spinningup.openai.com/), which offer both theoretical explanations of many deep RL algorithms and implementations that can be used with the Gym environments.